In [ ]:
!pip uninstall apache-beam -y && pip install -U pandas polars pyarrow narwhals 

In [ ]:
import pandas as pd
import polars as pl

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True

In [ ]:
from typing import Any
import narwhals as nw

def q13(
    customer_ds_raw: Any,
    orders_ds_raw: Any
) -> Any:

    customer_ds = nw.from_native(customer_ds_raw)
    orders_ds = nw.from_native(orders_ds_raw)
    
    var1 = "special"
    var2 = "requests"

    orders_ds = orders_ds.filter(~nw.col("o_comment").str.contains(f"{var1}.*{var2}"))
    
    result = (
        customer_ds.join(orders_ds, left_on="c_custkey", right_on="o_custkey", how="left")
        .group_by("c_custkey")
        .agg(nw.col("o_orderkey").len().alias("c_count"))
        .group_by("c_count")
        .agg(nw.col("c_count").len().alias("len"))
        .select(nw.col("c_count"), nw.col("len").alias("custdist"))
        .sort(by=["custdist", "c_count"], descending=[True, True])
    )

    return nw.to_native(result)

In [ ]:
dir_ = "/kaggle/input/tpc-h-data-parquet-s-2/"
customer = dir_ + 'customer.parquet'
orders = dir_ + 'orders.parquet'

In [ ]:
IO_FUNCS = {
    'pandas': lambda x: pd.read_parquet(x, engine='pyarrow'),
    'pandas[pyarrow]': lambda x: pd.read_parquet(x, engine='pyarrow', dtype_backend='pyarrow'),
    'polars[eager]': lambda x: pl.read_parquet(x),
    'polars[lazy]': lambda x: pl.scan_parquet(x),
}

In [ ]:
results = {}

## pandas via Narwhals

In [ ]:
tool = 'pandas'
fn = IO_FUNCS[tool]
timings = %timeit -o q13(fn(customer), fn(orders))
results[tool] = timings.all_runs

## pandas, pyarrow dtypes, via Narwhals

In [ ]:
tool = 'pandas[pyarrow]'
fn = IO_FUNCS[tool]
timings = %timeit -o q13(fn(customer), fn(orders))
results[tool] = timings.all_runs

## Polars read_parquet

In [ ]:
tool = 'polars[eager]'
fn = IO_FUNCS[tool]
timings = %timeit -o q13(fn(customer), fn(orders))
results[tool] = timings.all_runs

## Polars scan_parquet

In [ ]:
tool = 'polars[lazy]'
fn = IO_FUNCS[tool]
timings = %timeit -o q13(fn(customer), fn(orders)).collect()
results[tool] = timings.all_runs

## Save

In [ ]:
import json
with open('results.json', 'w') as fd:
    json.dump(results, fd)
